# Data Ingestion

In [1]:
import tensorflow as tf
import tfx
import tensorflow_model_analysis as tfma
import tensorflow_data_validation as tfdv

print('TF version: {}'.format(tf.__version__))
print('TFX version: {}'.format(tfx.__version__))
print('TFMA version: {}'.format(tfma.__version__))
print('TFDV version: {}'.format(tfdv.__version__))

TF version: 2.5.0
TFX version: 1.0.0
TFMA version: 0.30.0
TFDV version: 0.30.0


In [2]:
import os
import sys
import pprint
from tfx.components import CsvExampleGen
from tfx.proto import example_gen_pb2
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

In [3]:
tf.get_logger().propagate = False
tf.get_logger().setLevel('ERROR')
pp = pprint.PrettyPrinter()

In [4]:
pipeline_name = "consumer_complaint_pipeline_airflow"
base_dir = os.getcwd()
data_dir = os.path.join(base_dir, "data")
airflow_dir = os.path.join(base_dir, "airflow")
transform_file = os.path.join(base_dir, "transform.py")
tuner_file = os.path.join(base_dir, "tuner.py")
train_file = os.path.join(base_dir, "trainer.py")

print(data_dir)

Z:\tfx\data


In [5]:
pipeline_root = os.path.join(base_dir, "tfx")
metadata_path = os.path.join(pipeline_root, "metadata.sqlite")
serving_model_dir = os.path.join(pipeline_root, "serving_model")
print(pipeline_root)
print(metadata_path)
print(serving_model_dir)

Z:\tfx\tfx
Z:\tfx\tfx\metadata.sqlite
Z:\tfx\tfx\serving_model


In [6]:
context = InteractiveContext(pipeline_root='tfx')

In [7]:
input_config = example_gen_pb2.Input(splits=[
    example_gen_pb2.Input.Split(pattern='export-{SPAN}/*')
])

In [8]:
output_config = example_gen_pb2.Output(
        split_config=example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name='train', hash_buckets=6),
        example_gen_pb2.SplitConfig.Split(name='eval', hash_buckets=2),
        example_gen_pb2.SplitConfig.Split(name='test', hash_buckets=2)
    ]))

In [9]:
example_gen = CsvExampleGen(input_base=data_dir, output_config=output_config) #input_config=input_config, 

In [10]:
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 1
        type_id: 5
        uri: "tfx\\CsvExampleGen\\examples\\1"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\", \"test\"]"
          }
        }
        custom_properties {
          key: "input_fingerprint"
          value {
            string_value: "split:single_split,num_files:1,total_bytes:3396202,xor_checksum:1624628026,sum_checksum:1624628026"
          }
        }
        custom_properties {
          key: "payload_format"
          value {
            string_value: "FORMAT_TF_EXAMPLE"
          }
        }
        custom_properties {
          key: "span"
          value {
            int_value: 0
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.0.0"
          }
        }
        state: LIVE
        , artifact_type: id: 5
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [11]:
# get the artifact object
artifact = example_gen.outputs['examples'].get()[0]

# print split names and uri
print(f'split names: {artifact.split_names}')
print(f'artifact uri: {artifact.uri}')

split names: ["train", "eval", "test"]
artifact uri: tfx\CsvExampleGen\examples\1


# Data Validation

## Generate Statistics

In [12]:
from tfx.components import StatisticsGen

statistics_gen = StatisticsGen(
    examples=example_gen.outputs['examples'])

In [13]:
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(artifact: id: 2
        type_id: 7
        uri: "tfx\\StatisticsGen\\statistics\\2"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\", \"test\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "statistics"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "StatisticsGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.0.0"
          }
        }
        state: LIVE
        , artifact_type: id: 7
        name: "ExampleStatistics"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [14]:
context.show(statistics_gen.outputs['statistics'])

## Generate Schema 

In [15]:
from tfx.components import SchemaGen

schema_gen = SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    infer_feature_shape=True)

In [16]:
context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 3
        type_id: 9
        uri: "tfx\\SchemaGen\\schema\\3"
        custom_properties {
          key: "name"
          value {
            string_value: "schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "SchemaGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.0.0"
          }
        }
        state: LIVE
        , artifact_type: id: 9
        name: "Schema"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [17]:
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'age',INT,required,,-
'capital-gain',INT,required,,-
'capital-loss',INT,required,,-
'education',STRING,required,,'education'
'education-num',INT,required,,-
'fnlwgt',INT,required,,-
'hours-per-week',INT,required,,-
'label',INT,required,,-
'marital-status',STRING,required,,'marital-status'


Z:\ProgramData\Anaconda3\envs\tfx_env\lib\site-packages\tensorflow_data_validation\utils\display_util.py:180: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


,Values
Domain,
'education',"'10th', '11th', '12th', '1st-4th', '5th-6th', '7th-8th', '9th', 'Assoc-acdm', 'Assoc-voc', 'Bachelors', 'Doctorate', 'HS-grad', 'Masters', 'Preschool', 'Prof-school', 'Some-college'"
'marital-status',"'Divorced', 'Married-AF-spouse', 'Married-civ-spouse', 'Married-spouse-absent', 'Never-married', 'Separated', 'Widowed'"
'native-country',"'?', 'Cambodia', 'Canada', 'China', 'Columbia', 'Cuba', 'Dominican-Republic', 'Ecuador', 'El-Salvador', 'England', 'France', 'Germany', 'Greece', 'Guatemala', 'Haiti', 'Holand-Netherlands', 'Honduras', 'Hong', 'Hungary', 'India', 'Iran', 'Ireland', 'Italy', 'Jamaica', 'Japan', 'Laos', 'Mexico', 'Nicaragua', 'Outlying-US(Guam-USVI-etc)', 'Peru', 'Philippines', 'Poland', 'Portugal', 'Puerto-Rico', 'Scotland', 'South', 'Taiwan', 'Thailand', 'Trinadad&Tobago', 'United-States', 'Vietnam', 'Yugoslavia'"
'occupation',"'?', 'Adm-clerical', 'Armed-Forces', 'Craft-repair', 'Exec-managerial', 'Farming-fishing', 'Handlers-cleaners', 'Machine-op-inspct', 'Other-service', 'Priv-house-serv', 'Prof-specialty', 'Protective-serv', 'Sales', 'Tech-support', 'Transport-moving'"
'race',"'Amer-Indian-Eskimo', 'Asian-Pac-Islander', 'Black', 'Other', 'White'"
'relationship',"'Husband', 'Not-in-family', 'Other-relative', 'Own-child', 'Unmarried', 'Wife'"
'sex',"'Female', 'Male'"
'workclass',"'?', 'Federal-gov', 'Local-gov', 'Never-worked', 'Private', 'Self-emp-inc', 'Self-emp-not-inc', 'State-gov', 'Without-pay'"


## Validate Data

In [18]:
from tfx.components import ExampleValidator

example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema'])

In [19]:
context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: Channel(
            type_name: ExampleAnomalies
            artifacts: [Artifact(artifact: id: 4
        type_id: 11
        uri: "tfx\\ExampleValidator\\anomalies\\4"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\", \"test\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "anomalies"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "ExampleValidator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.0.0"
          }
        }
        state: LIVE
        , artifact_type: id: 11
        name: "ExampleAnomalies"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [20]:
context.show(example_validator.outputs['anomalies'])

Z:\ProgramData\Anaconda3\envs\tfx_env\lib\site-packages\tensorflow_data_validation\utils\display_util.py:217: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


# Transform (Preprocess) Data

In [21]:
# Set the constants module filename
_census_constants_module_file = 'census_constants.py'

In [22]:
%%writefile {_census_constants_module_file}

# Features with string data types that will be converted to indices
VOCAB_FEATURE_DICT = {
    'education': 16, 'marital-status': 7, 'occupation': 15, 'race': 5, 
    'relationship': 6, 'workclass': 9, 'sex': 2, 'native-country': 42
}

# Numerical features that are marked as continuous
NUMERIC_FEATURE_KEYS = ['fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']

# Feature that can be grouped into buckets
BUCKET_FEATURE_DICT = {'age': 4}

# Number of out-of-vocabulary buckets
NUM_OOV_BUCKETS = 5

# Feature that the model will predict
LABEL_KEY = 'label'

# Utility function for renaming the feature
def transformed_name(key):
    return key + '_xf'

Overwriting census_constants.py


In [23]:
transform_file = 'transform.py'

In [24]:
%%writefile {transform_file}

import tensorflow as tf
import tensorflow_transform as tft

# import constants from cells above
import census_constants

# Unpack the contents of the constants module
_NUMERIC_FEATURE_KEYS = census_constants.NUMERIC_FEATURE_KEYS
_VOCAB_FEATURE_DICT = census_constants.VOCAB_FEATURE_DICT
_BUCKET_FEATURE_DICT = census_constants.BUCKET_FEATURE_DICT
_NUM_OOV_BUCKETS = census_constants.NUM_OOV_BUCKETS
_LABEL_KEY = census_constants.LABEL_KEY
_transformed_name = census_constants.transformed_name


def preprocessing_fn(inputs):
    outputs = {}

    # Scale these features to the range [0,1]
    for key in _NUMERIC_FEATURE_KEYS:
        outputs[_transformed_name(key)] = tft.scale_to_0_1(inputs[key])
    
    # Bucketize these features
    for key, num_buckets in _BUCKET_FEATURE_DICT.items():
        #outputs[transformed_name(key)] = tft.bucketize(inputs[key], FEATURE_BUCKET_COUNT[key])
        indices = tft.bucketize(inputs[key], num_buckets)
        one_hot = tf.one_hot(indices, num_buckets)
        outputs[_transformed_name(key)] = tf.reshape(one_hot, [-1, num_buckets])
            #always_return_num_quantiles=False)

    # Convert strings to indices in a vocabulary
    for key, vocab_size in _VOCAB_FEATURE_DICT.items():
        #outputs[transformed_name(key)] = tft.compute_and_apply_vocabulary(inputs[key])
        indices = tft.compute_and_apply_vocabulary(inputs[key])
        one_hot = tf.one_hot(indices, vocab_size)
        outputs[_transformed_name(key)] = tf.reshape(one_hot, [-1, vocab_size])

    # Convert the label strings to an index
    outputs[_transformed_name(_LABEL_KEY)] = tft.compute_and_apply_vocabulary(inputs[_LABEL_KEY])

    return outputs

Overwriting transform.py


In [25]:
from tfx.components import Transform

transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=transform_file)

In [26]:
context.run(transform)

ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: Channel(
            type_name: TransformGraph
            artifacts: [Artifact(artifact: id: 5
        type_id: 13
        uri: "tfx\\Transform\\transform_graph\\5"
        custom_properties {
          key: "name"
          value {
            string_value: "transform_graph"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.0.0"
          }
        }
        state: LIVE
        , artifact_type: id: 13
        name: "TransformGraph"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        transformed_examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 6
        type_id: 5
        uri: "tfx\\Transform\\transformed_examples\\5"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\", \"test\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "transformed_examples"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.0.0"
          }
        }
        state: LIVE
        , artifact_type: id: 5
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        updated_analyzer_cache: Channel(
            type_name: TransformCache
            artifacts: [Artifact(artifact: id: 7
        type_id: 14
        uri: "tfx\\Transform\\updated_analyzer_cache\\5"
        custom_properties {
          key: "name"
          value {
            string_value: "updated_analyzer_cache"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.0.0"
          }
        }
        state: LIVE
        , artifact_type: id: 14
        name: "TransformCache"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        pre_transform_schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 8
        type_id: 9
        uri: "tfx\\Transform\\pre_transform_schema\\5"
        custom_properties {
          key: "name"
          value {
            string_value: "pre_transform_schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.0.0"
          }
        }
        state: LIVE
        , artifact_t

# Tuner

In [27]:
# Declare name of module file
tuner_file = 'tuner.py'

In [28]:
%%writefile {tuner_file}

# Define imports
from kerastuner.engine import base_tuner
import kerastuner as kt
from tensorflow import keras
from typing import NamedTuple, Dict, Text, Any, List
from tfx.components.trainer.fn_args_utils import FnArgs, DataAccessor
import tensorflow as tf
import tensorflow_transform as tft

# import same constants from transform module
import census_constants

# Unpack the contents of the constants module
_NUMERIC_FEATURE_KEYS = census_constants.NUMERIC_FEATURE_KEYS
_VOCAB_FEATURE_DICT = census_constants.VOCAB_FEATURE_DICT
_BUCKET_FEATURE_DICT = census_constants.BUCKET_FEATURE_DICT
_NUM_OOV_BUCKETS = census_constants.NUM_OOV_BUCKETS
_LABEL_KEY = census_constants.LABEL_KEY
_transformed_name = census_constants.transformed_name

TunerFnResult = NamedTuple('TunerFnResult', [('tuner', base_tuner.BaseTuner),
                                             ('fit_kwargs', Dict[Text, Any])])

# Callback for the search strategy
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)


def _gzip_reader_fn(filenames):
    # Load the dataset. Specify the compression type since it is saved as `.gz`
    return tf.data.TFRecordDataset(filenames, compression_type='GZIP')

def _input_fn(file_pattern,
              tf_transform_output,
              num_epochs=None,
              batch_size=32) -> tf.data.Dataset:
    
    transformed_feature_spec = (
      tf_transform_output.transformed_feature_spec().copy())
    
    # Create batches of features and labels
    dataset = tf.data.experimental.make_batched_features_dataset(
      file_pattern=file_pattern,
      batch_size=batch_size,
      features=transformed_feature_spec,
      reader=_gzip_reader_fn,
      num_epochs=num_epochs,
      label_key=_transformed_name(_LABEL_KEY))
    
    return dataset


def _make_keras_model(hp) -> tf.keras.Model:
    # Define input layers for numeric keys
    input_numeric = [
        tf.keras.layers.Input(name=_transformed_name(colname), shape=(1,), dtype=tf.float32)
        for colname in _NUMERIC_FEATURE_KEYS
    ]
    
    # Define input layers for vocab keys
    input_categorical = [
        tf.keras.layers.Input(name=_transformed_name(colname), shape=(vocab_size,), dtype=tf.float32)
        for colname, vocab_size in _VOCAB_FEATURE_DICT.items()
    ]
    
    # Define input layers for bucket key
    input_categorical += [
        tf.keras.layers.Input(name=_transformed_name(colname), shape=(num_buckets,), dtype=tf.float32)
        for colname, num_buckets in _BUCKET_FEATURE_DICT.items()
    ]
    
    # Concatenate numeric inputs
    deep = tf.keras.layers.concatenate(input_numeric)
    
    # Create deep dense network for numeric inputs
    hp_units = hp.Int('units', min_value=32, max_value=512, step=16)
    for _ in range(4):
        deep = keras.layers.Dense(hp_units, activation='relu')(deep)
    
    # Concatenate categorical inputs
    wide = tf.keras.layers.concatenate(input_categorical)
    
    # Combine wide and deep networks
    combined = tf.keras.layers.concatenate([deep, wide])
    
    # Define output of binary classifier
    output = tf.keras.layers.Dense(1, activation='sigmoid')(combined)
    
    # Setup combined input
    input_layers = input_numeric + input_categorical
    
    # Create the Keras model
    model = tf.keras.Model(input_layers, output)
    
    # Define training parameters
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    model.compile(
        loss='binary_crossentropy',
        #optimizer=tf.keras.optimizers.Adam(lr=0.001),
        optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),
        metrics='accuracy')
    
    return model
    

def tuner_fn(fn_args: FnArgs) -> TunerFnResult:
    
    # Define tuner search strategy
    tuner = kt.Hyperband(_make_keras_model,
                     objective='accuracy', 
                     max_epochs=50,
                     factor=3,
                     directory=fn_args.working_dir,
                     project_name='kt_hyperband')
    
    # Load transform output
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)
        
    # Use _input_fn() to extract input features and labels from the train and val set
    train_set = _input_fn(fn_args.train_files[0], tf_transform_output)
    val_set = _input_fn(fn_args.eval_files[0], tf_transform_output)
    
    
    return TunerFnResult(
      tuner=tuner,
      fit_kwargs={ 
          "callbacks":[stop_early],
          'x': train_set,
          'validation_data': val_set,
          'steps_per_epoch': fn_args.train_steps,
          'validation_steps': fn_args.eval_steps
      }
    )

Overwriting tuner.py


In [29]:
TRAIN_STEPS = 500 #50000
EVAL_STEPS = 100 #10000

In [30]:
from tfx.proto import trainer_pb2
from tfx.components import Tuner

# Setup the Tuner component
tuner = Tuner(
    module_file=tuner_file,
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(splits=['train'], num_steps=TRAIN_STEPS),
    eval_args=trainer_pb2.EvalArgs(splits=['eval'], num_steps=EVAL_STEPS)
    )

In [31]:
context.run(tuner, enable_cache=False)

Epoch 1/2
500/500 [==============================] - ETA: 10:04 - loss: 0.6408 - accuracy: 0.750 - ETA: 6s - loss: 0.6613 - accuracy: 0.7125  - ETA: 5s - loss: 0.6075 - accuracy: 0.75 - ETA: 6s - loss: 0.5706 - accuracy: 0.75 - ETA: 6s - loss: 0.5598 - accuracy: 0.74 - ETA: 6s - loss: 0.5447 - accuracy: 0.74 - ETA: 6s - loss: 0.5250 - accuracy: 0.75 - ETA: 6s - loss: 0.5148 - accuracy: 0.75 - ETA: 6s - loss: 0.5074 - accuracy: 0.76 - ETA: 7s - loss: 0.5012 - accuracy: 0.76 - ETA: 7s - loss: 0.4982 - accuracy: 0.76 - ETA: 7s - loss: 0.4928 - accuracy: 0.76 - ETA: 7s - loss: 0.4890 - accuracy: 0.76 - ETA: 7s - loss: 0.4817 - accuracy: 0.77 - ETA: 7s - loss: 0.4752 - accuracy: 0.77 - ETA: 7s - loss: 0.4703 - accuracy: 0.77 - ETA: 7s - loss: 0.4650 - accuracy: 0.78 - ETA: 7s - loss: 0.4583 - accuracy: 0.78 - ETA: 7s - loss: 0.4505 - accuracy: 0.79 - ETA: 6s - loss: 0.4414 - accuracy: 0.79 - ETA: 6s - loss: 0.4345 - accuracy: 0.79 - ETA: 6s - loss: 0.4294 - accuracy: 0.79 - ETA: 6s - loss: 

500/500 [==============================] - ETA: 5s - loss: 0.2997 - accuracy: 0.90 - ETA: 5s - loss: 0.3540 - accuracy: 0.84 - ETA: 4s - loss: 0.3232 - accuracy: 0.85 - ETA: 4s - loss: 0.3293 - accuracy: 0.84 - ETA: 4s - loss: 0.3295 - accuracy: 0.85 - ETA: 5s - loss: 0.3317 - accuracy: 0.85 - ETA: 5s - loss: 0.3291 - accuracy: 0.85 - ETA: 5s - loss: 0.3323 - accuracy: 0.84 - ETA: 5s - loss: 0.3397 - accuracy: 0.84 - ETA: 6s - loss: 0.3309 - accuracy: 0.85 - ETA: 6s - loss: 0.3392 - accuracy: 0.84 - ETA: 6s - loss: 0.3365 - accuracy: 0.85 - ETA: 6s - loss: 0.3359 - accuracy: 0.85 - ETA: 6s - loss: 0.3339 - accuracy: 0.85 - ETA: 6s - loss: 0.3385 - accuracy: 0.85 - ETA: 5s - loss: 0.3386 - accuracy: 0.85 - ETA: 5s - loss: 0.3373 - accuracy: 0.85 - ETA: 5s - loss: 0.3366 - accuracy: 0.85 - ETA: 5s - loss: 0.3348 - accuracy: 0.85 - ETA: 5s - loss: 0.3309 - accuracy: 0.85 - ETA: 5s - loss: 0.3343 - accuracy: 0.85 - ETA: 5s - loss: 0.3333 - accuracy: 0.85 - ETA: 5s - loss: 0.3356 - accuracy

Epoch 1/2
500/500 [==============================] - ETA: 5:35 - loss: 0.6141 - accuracy: 0.81 - ETA: 4s - loss: 0.6313 - accuracy: 0.6927 - ETA: 5s - loss: 0.6261 - accuracy: 0.69 - ETA: 6s - loss: 0.6158 - accuracy: 0.70 - ETA: 6s - loss: 0.5903 - accuracy: 0.73 - ETA: 7s - loss: 0.5843 - accuracy: 0.73 - ETA: 7s - loss: 0.5714 - accuracy: 0.74 - ETA: 6s - loss: 0.5760 - accuracy: 0.73 - ETA: 6s - loss: 0.5706 - accuracy: 0.74 - ETA: 6s - loss: 0.5589 - accuracy: 0.74 - ETA: 6s - loss: 0.5598 - accuracy: 0.74 - ETA: 6s - loss: 0.5578 - accuracy: 0.74 - ETA: 5s - loss: 0.5532 - accuracy: 0.74 - ETA: 5s - loss: 0.5449 - accuracy: 0.75 - ETA: 5s - loss: 0.5380 - accuracy: 0.75 - ETA: 5s - loss: 0.5323 - accuracy: 0.75 - ETA: 5s - loss: 0.5281 - accuracy: 0.76 - ETA: 5s - loss: 0.5252 - accuracy: 0.76 - ETA: 5s - loss: 0.5223 - accuracy: 0.76 - ETA: 5s - loss: 0.5212 - accuracy: 0.76 - ETA: 5s - loss: 0.5192 - accuracy: 0.76 - ETA: 5s - loss: 0.5172 - accuracy: 0.76 - ETA: 5s - loss: 0.5

500/500 [==============================] - ETA: 10s - loss: 0.3114 - accuracy: 0.875 - ETA: 3s - loss: 0.2981 - accuracy: 0.878 - ETA: 3s - loss: 0.3365 - accuracy: 0.85 - ETA: 3s - loss: 0.3383 - accuracy: 0.84 - ETA: 3s - loss: 0.3468 - accuracy: 0.84 - ETA: 3s - loss: 0.3541 - accuracy: 0.84 - ETA: 3s - loss: 0.3528 - accuracy: 0.84 - ETA: 3s - loss: 0.3521 - accuracy: 0.84 - ETA: 3s - loss: 0.3544 - accuracy: 0.83 - ETA: 3s - loss: 0.3535 - accuracy: 0.83 - ETA: 4s - loss: 0.3551 - accuracy: 0.83 - ETA: 4s - loss: 0.3541 - accuracy: 0.83 - ETA: 4s - loss: 0.3532 - accuracy: 0.83 - ETA: 4s - loss: 0.3523 - accuracy: 0.83 - ETA: 4s - loss: 0.3498 - accuracy: 0.84 - ETA: 4s - loss: 0.3542 - accuracy: 0.83 - ETA: 4s - loss: 0.3540 - accuracy: 0.83 - ETA: 4s - loss: 0.3525 - accuracy: 0.83 - ETA: 4s - loss: 0.3515 - accuracy: 0.84 - ETA: 4s - loss: 0.3499 - accuracy: 0.84 - ETA: 4s - loss: 0.3510 - accuracy: 0.83 - ETA: 4s - loss: 0.3528 - accuracy: 0.83 - ETA: 4s - loss: 0.3555 - accur

Epoch 1/2
500/500 [==============================] - ETA: 6:01 - loss: 0.7148 - accuracy: 0.43 - ETA: 4s - loss: 0.6803 - accuracy: 0.5580 - ETA: 5s - loss: 0.6126 - accuracy: 0.67 - ETA: 5s - loss: 0.6014 - accuracy: 0.70 - ETA: 5s - loss: 0.5805 - accuracy: 0.73 - ETA: 5s - loss: 0.5957 - accuracy: 0.72 - ETA: 5s - loss: 0.5897 - accuracy: 0.73 - ETA: 5s - loss: 0.5956 - accuracy: 0.72 - ETA: 5s - loss: 0.5930 - accuracy: 0.73 - ETA: 4s - loss: 0.5886 - accuracy: 0.73 - ETA: 4s - loss: 0.5799 - accuracy: 0.74 - ETA: 4s - loss: 0.5716 - accuracy: 0.74 - ETA: 4s - loss: 0.5681 - accuracy: 0.74 - ETA: 4s - loss: 0.5595 - accuracy: 0.75 - ETA: 4s - loss: 0.5502 - accuracy: 0.75 - ETA: 3s - loss: 0.5455 - accuracy: 0.76 - ETA: 3s - loss: 0.5439 - accuracy: 0.76 - ETA: 3s - loss: 0.5415 - accuracy: 0.76 - ETA: 3s - loss: 0.5353 - accuracy: 0.76 - ETA: 3s - loss: 0.5338 - accuracy: 0.76 - ETA: 3s - loss: 0.5266 - accuracy: 0.77 - ETA: 3s - loss: 0.5240 - accuracy: 0.77 - ETA: 3s - loss: 0.5

...

500/500 [==============================] - ETA: 4s - loss: 0.2798 - accuracy: 0.93 - ETA: 3s - loss: 0.3103 - accuracy: 0.86 - ETA: 3s - loss: 0.3082 - accuracy: 0.86 - ETA: 3s - loss: 0.3145 - accuracy: 0.86 - ETA: 3s - loss: 0.3029 - accuracy: 0.86 - ETA: 3s - loss: 0.3032 - accuracy: 0.86 - ETA: 3s - loss: 0.2987 - accuracy: 0.86 - ETA: 3s - loss: 0.3064 - accuracy: 0.86 - ETA: 3s - loss: 0.3025 - accuracy: 0.86 - ETA: 3s - loss: 0.3030 - accuracy: 0.86 - ETA: 3s - loss: 0.3012 - accuracy: 0.86 - ETA: 3s - loss: 0.2991 - accuracy: 0.86 - ETA: 3s - loss: 0.3000 - accuracy: 0.86 - ETA: 3s - loss: 0.3021 - accuracy: 0.86 - ETA: 3s - loss: 0.3003 - accuracy: 0.86 - ETA: 3s - loss: 0.2977 - accuracy: 0.86 - ETA: 3s - loss: 0.3006 - accuracy: 0.86 - ETA: 3s - loss: 0.3046 - accuracy: 0.85 - ETA: 2s - loss: 0.3049 - accuracy: 0.85 - ETA: 2s - loss: 0.3047 - accuracy: 0.85 - ETA: 2s - loss: 0.3053 - accuracy: 0.85 - ETA: 2s - loss: 0.3059 - accuracy: 0.85 - ETA: 2s - loss: 0.3065 - accuracy

500/500 [==============================] - ETA: 5s - loss: 0.4910 - accuracy: 0.71 - ETA: 3s - loss: 0.3435 - accuracy: 0.81 - ETA: 3s - loss: 0.3367 - accuracy: 0.84 - ETA: 3s - loss: 0.3273 - accuracy: 0.84 - ETA: 3s - loss: 0.3261 - accuracy: 0.84 - ETA: 3s - loss: 0.3189 - accuracy: 0.84 - ETA: 3s - loss: 0.3098 - accuracy: 0.85 - ETA: 3s - loss: 0.3086 - accuracy: 0.85 - ETA: 3s - loss: 0.3185 - accuracy: 0.85 - ETA: 3s - loss: 0.3220 - accuracy: 0.84 - ETA: 3s - loss: 0.3270 - accuracy: 0.84 - ETA: 3s - loss: 0.3317 - accuracy: 0.84 - ETA: 3s - loss: 0.3312 - accuracy: 0.84 - ETA: 3s - loss: 0.3307 - accuracy: 0.84 - ETA: 3s - loss: 0.3331 - accuracy: 0.84 - ETA: 3s - loss: 0.3304 - accuracy: 0.84 - ETA: 3s - loss: 0.3287 - accuracy: 0.84 - ETA: 3s - loss: 0.3277 - accuracy: 0.84 - ETA: 3s - loss: 0.3261 - accuracy: 0.84 - ETA: 3s - loss: 0.3270 - accuracy: 0.84 - ETA: 3s - loss: 0.3256 - accuracy: 0.84 - ETA: 2s - loss: 0.3230 - accuracy: 0.84 - ETA: 2s - loss: 0.3227 - accuracy

500/500 [==============================] - ETA: 4s - loss: 0.2100 - accuracy: 0.87 - ETA: 3s - loss: 0.3145 - accuracy: 0.86 - ETA: 3s - loss: 0.2917 - accuracy: 0.86 - ETA: 3s - loss: 0.2989 - accuracy: 0.86 - ETA: 3s - loss: 0.2945 - accuracy: 0.87 - ETA: 3s - loss: 0.3023 - accuracy: 0.87 - ETA: 3s - loss: 0.3057 - accuracy: 0.86 - ETA: 3s - loss: 0.3081 - accuracy: 0.86 - ETA: 3s - loss: 0.3178 - accuracy: 0.85 - ETA: 3s - loss: 0.3218 - accuracy: 0.85 - ETA: 3s - loss: 0.3197 - accuracy: 0.85 - ETA: 3s - loss: 0.3220 - accuracy: 0.85 - ETA: 3s - loss: 0.3172 - accuracy: 0.85 - ETA: 3s - loss: 0.3156 - accuracy: 0.85 - ETA: 3s - loss: 0.3159 - accuracy: 0.85 - ETA: 3s - loss: 0.3169 - accuracy: 0.85 - ETA: 3s - loss: 0.3203 - accuracy: 0.85 - ETA: 3s - loss: 0.3192 - accuracy: 0.85 - ETA: 3s - loss: 0.3204 - accuracy: 0.85 - ETA: 3s - loss: 0.3195 - accuracy: 0.85 - ETA: 3s - loss: 0.3157 - accuracy: 0.85 - ETA: 3s - loss: 0.3160 - accuracy: 0.85 - ETA: 2s - loss: 0.3162 - accuracy

500/500 [==============================] - ETA: 4s - loss: 0.3277 - accuracy: 0.84 - ETA: 3s - loss: 0.3159 - accuracy: 0.86 - ETA: 3s - loss: 0.3331 - accuracy: 0.84 - ETA: 3s - loss: 0.3252 - accuracy: 0.85 - ETA: 3s - loss: 0.3226 - accuracy: 0.85 - ETA: 3s - loss: 0.3163 - accuracy: 0.85 - ETA: 3s - loss: 0.3127 - accuracy: 0.85 - ETA: 3s - loss: 0.3106 - accuracy: 0.85 - ETA: 3s - loss: 0.3083 - accuracy: 0.85 - ETA: 3s - loss: 0.3110 - accuracy: 0.85 - ETA: 3s - loss: 0.3144 - accuracy: 0.85 - ETA: 3s - loss: 0.3131 - accuracy: 0.85 - ETA: 3s - loss: 0.3143 - accuracy: 0.85 - ETA: 3s - loss: 0.3138 - accuracy: 0.85 - ETA: 3s - loss: 0.3165 - accuracy: 0.85 - ETA: 3s - loss: 0.3165 - accuracy: 0.85 - ETA: 3s - loss: 0.3175 - accuracy: 0.85 - ETA: 3s - loss: 0.3174 - accuracy: 0.85 - ETA: 3s - loss: 0.3176 - accuracy: 0.85 - ETA: 3s - loss: 0.3187 - accuracy: 0.85 - ETA: 3s - loss: 0.3164 - accuracy: 0.85 - ETA: 3s - loss: 0.3182 - accuracy: 0.85 - ETA: 3s - loss: 0.3172 - accuracy

ExecutionResult(
    component_id: Tuner
    execution_id: 6
    outputs:
        best_hyperparameters: Channel(
            type_name: HyperParameters
            artifacts: [Artifact(artifact: id: 13
        type_id: 16
        uri: "tfx\\Tuner\\best_hyperparameters\\6"
        custom_properties {
          key: "name"
          value {
            string_value: "best_hyperparameters"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Tuner"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.0.0"
          }
        }
        state: LIVE
        , artifact_type: id: 16
        name: "HyperParameters"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

# Trainer

In [32]:
# Declare trainer module file
_trainer_module_file = 'trainer.py'

In [33]:
%%writefile {_trainer_module_file}

import os
from tensorflow import keras
from typing import NamedTuple, Dict, Text, Any, List
from tfx.components.trainer.fn_args_utils import FnArgs, DataAccessor
import tensorflow as tf
import tensorflow_transform as tft

from tensorflow_transform.tf_metadata import schema_utils
from tfx_bsl.public.tfxio import TensorFlowDatasetOptions

# import same constants from transform module
import census_constants

# Unpack the contents of the constants module
_NUMERIC_FEATURE_KEYS = census_constants.NUMERIC_FEATURE_KEYS
_VOCAB_FEATURE_DICT = census_constants.VOCAB_FEATURE_DICT
_BUCKET_FEATURE_DICT = census_constants.BUCKET_FEATURE_DICT
_NUM_OOV_BUCKETS = census_constants.NUM_OOV_BUCKETS
_LABEL_KEY = census_constants.LABEL_KEY
_transformed_name = census_constants.transformed_name

def _gzip_reader_fn(filenames):  
    # Load the dataset. Specify the compression type since it is saved as `.gz`
    return tf.data.TFRecordDataset(filenames, compression_type='GZIP')

def _input_fn(file_pattern,
              tf_transform_output,
              num_epochs=None,
              batch_size=32) -> tf.data.Dataset:
    
    transformed_feature_spec = (
      tf_transform_output.transformed_feature_spec().copy())
    
    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transformed_feature_spec,
        reader=_gzip_reader_fn,
        num_epochs=num_epochs,
        label_key=_transformed_name(_LABEL_KEY)
    )
    
    return dataset

def _get_serve_tf_examples_fn(model, tf_transform_output):
    
    # Get transformation graph
    model.tft_layer = tf_transform_output.transform_features_layer()
       
    @tf.function
    def serve_tf_examples_fn(serialized_tf_examples):
        # Get pre-transform feature spec
        feature_spec = tf_transform_output.raw_feature_spec()

        # Pop label since serving inputs do not include the label
        feature_spec.pop(_LABEL_KEY)

        # Parse raw examples into a dictionary of tensors matching the feature spec
        parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)

        # Transform the raw examples using the transform graph
        transformed_features = model.tft_layer(parsed_features)

        # Get predictions using the transformed features
        return model(transformed_features)

    return serve_tf_examples_fn


def _make_keras_model(hp) -> tf.keras.Model:
    # Define input layers for numeric keys
    input_numeric = [
        tf.keras.layers.Input(name=_transformed_name(colname), shape=(1,), dtype=tf.float32)
        for colname in _NUMERIC_FEATURE_KEYS
    ]
    
    # Define input layers for vocab keys
    input_categorical = [
        tf.keras.layers.Input(name=_transformed_name(colname), shape=(vocab_size,), dtype=tf.float32)
        for colname, vocab_size in _VOCAB_FEATURE_DICT.items()
    ]
    
    # Define input layers for bucket key
    input_categorical += [
        tf.keras.layers.Input(name=_transformed_name(colname), shape=(num_buckets,), dtype=tf.float32)
        for colname, num_buckets in _BUCKET_FEATURE_DICT.items()
    ]
    
    # Concatenate numeric inputs
    deep = tf.keras.layers.concatenate(input_numeric)
    
    # Create deep dense network for numeric inputs
    hp_units = hp.get('units')
    for _ in range(4):
        deep = keras.layers.Dense(hp_units, activation='relu')(deep)
    
    # Concatenate categorical inputs
    wide = tf.keras.layers.concatenate(input_categorical)
    
    # Combine wide and deep networks
    combined = tf.keras.layers.concatenate([deep, wide])
    
    # Define output of binary classifier
    output = tf.keras.layers.Dense(1, activation='sigmoid')(combined)
    
    # Setup combined input
    input_layers = input_numeric + input_categorical
    
    # Create the Keras model
    model = tf.keras.Model(input_layers, output)
    
    # Define training parameters
    hp_learning_rate = hp.get('learning_rate')
    model.compile(
        loss='binary_crossentropy',
        optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),
        metrics='accuracy')
    
    return model
        
def run_fn(fn_args: FnArgs) -> None:
    
    # Callback for TensorBoard
    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir=fn_args.model_run_dir, 
        update_freq='batch')
    
    # Load transform output
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)
  
    # Create batches of data good for 10 epochs
    train_set = _input_fn(fn_args.train_files[0], tf_transform_output, 10)
    val_set = _input_fn(fn_args.eval_files[0], tf_transform_output, 10)
    
    # Load best hyperparameters
    hp = fn_args.hyperparameters.get('values')
    
    # Build the model
    model = _make_keras_model(hp) #model_builder()
    
    # Train the model
    
    model.fit(
        x=train_set,
        steps_per_epoch=fn_args.train_steps,
        validation_data=val_set,
        validation_steps=fn_args.eval_steps,
        callbacks=[tensorboard_callback]
      )
    
    # Define default serving signature
    signatures = {
      'serving_default':
          _get_serve_tf_examples_fn(model,
                                    tf_transform_output).get_concrete_function(
                                        tf.TensorSpec(
                                            shape=[None],
                                            dtype=tf.string,
                                            name='examples')),
    }

    # Save the model
    model.save(fn_args.serving_model_dir, save_format='tf', signatures=signatures)

Overwriting trainer.py


In [34]:
from tfx.components import Trainer

trainer = Trainer(
    module_file=_trainer_module_file,
    examples=transform.outputs['transformed_examples'],
    hyperparameters=tuner.outputs['best_hyperparameters'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(splits=['train'], num_steps=TRAIN_STEPS),
    eval_args=trainer_pb2.EvalArgs(splits=['eval'], num_steps=EVAL_STEPS)
)

In [35]:
context.run(trainer, enable_cache=False)

500/500 [==============================] - ETA: 6:02 - loss: 0.6418 - accuracy: 0.78 - ETA: 24s - loss: 0.6127 - accuracy: 0.7812 - ETA: 12s - loss: 0.6133 - accuracy: 0.758 - ETA: 9s - loss: 0.5856 - accuracy: 0.765 - ETA: 7s - loss: 0.5770 - accuracy: 0.76 - ETA: 7s - loss: 0.5667 - accuracy: 0.77 - ETA: 7s - loss: 0.5696 - accuracy: 0.76 - ETA: 7s - loss: 0.5568 - accuracy: 0.77 - ETA: 7s - loss: 0.5542 - accuracy: 0.77 - ETA: 7s - loss: 0.5472 - accuracy: 0.77 - ETA: 7s - loss: 0.5406 - accuracy: 0.77 - ETA: 7s - loss: 0.5413 - accuracy: 0.76 - ETA: 7s - loss: 0.5352 - accuracy: 0.77 - ETA: 7s - loss: 0.5334 - accuracy: 0.76 - ETA: 7s - loss: 0.5287 - accuracy: 0.76 - ETA: 6s - loss: 0.5296 - accuracy: 0.76 - ETA: 6s - loss: 0.5266 - accuracy: 0.76 - ETA: 6s - loss: 0.5207 - accuracy: 0.77 - ETA: 6s - loss: 0.5186 - accuracy: 0.77 - ETA: 6s - loss: 0.5161 - accuracy: 0.77 - ETA: 6s - loss: 0.5143 - accuracy: 0.77 - ETA: 6s - loss: 0.5102 - accuracy: 0.77 - ETA: 6s - loss: 0.5036 - 

ExecutionResult(
    component_id: Trainer
    execution_id: 7
    outputs:
        model: Channel(
            type_name: Model
            artifacts: [Artifact(artifact: id: 14
        type_id: 18
        uri: "tfx\\Trainer\\model\\7"
        custom_properties {
          key: "name"
          value {
            string_value: "model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.0.0"
          }
        }
        state: LIVE
        , artifact_type: id: 18
        name: "Model"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        model_run: Channel(
            type_name: ModelRun
            artifacts: [Artifact(artifact: id: 15
        type_id: 19
        uri: "tfx\\Trainer\\model_run\\7"
        custom_properties {
          key: "name"
          value {
            string_value: "model_run"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.0.0"
          }
        }
        state: LIVE
        , artifact_type: id: 19
        name: "ModelRun"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [36]:
# Get `model` output of the component
model_artifact_dir = trainer.outputs['model'].get()[0].uri

# List top-level directory
pp.pprint(os.listdir(model_artifact_dir))

# Get model directory
model_dir = os.path.join(model_artifact_dir, 'Format-Serving')

# List subdirectories
pp.pprint(os.listdir(model_dir))

['Format-Serving']
['assets', 'keras_metadata.pb', 'saved_model.pb', 'variables']


In [37]:
model_run_artifact_dir = trainer.outputs['model_run'].get()[0].uri
model_dir = trainer.outputs['model'].get()[0].uri

%load_ext tensorboard
%tensorboard --logdir {model_dir}

Reusing TensorBoard on port 6006 (pid 8740), started 4:48:32 ago. (Use '!kill 8740' to kill it.)

# Evaluation

## Define EvalConfig

In [38]:
import tensorflow_model_analysis as tfma
from google.protobuf import text_format

eval_config_1 = text_format.Parse("""
  ## Model information
  model_specs {
    # This assumes a serving model with signature 'serving_default'.
    label_key: "label"
  }

  ## Post training metric information
  metrics_specs {
    metrics { class_name: "ExampleCount" }
    metrics {
      class_name: "BinaryAccuracy"
      threshold {
        # Ensure that metric is always > 0.5
        value_threshold {
          lower_bound { value: 0.65 }
        }
        # Ensure that metric does not drop by more than a small epsilon
        # e.g. (candidate - baseline) > -1e-10 or candidate > baseline - 1e-10
        change_threshold {
          direction: HIGHER_IS_BETTER
          absolute { value: -1e-10 }
        }
      }
    }
    metrics { class_name: "BinaryCrossentropy" }
    metrics { class_name: "AUC" }
    metrics { class_name: "AUCPrecisionRecall" }
    metrics { class_name: "Precision" }
    metrics { class_name: "Recall" }
    metrics { class_name: "MeanLabel" }
    metrics { class_name: "MeanPrediction" }
    metrics { class_name: "Calibration" }
    metrics { class_name: "CalibrationPlot" }
    metrics { class_name: "ConfusionMatrixPlot" }
    # ... add additional metrics and plots ...
  }

  ## Slicing information
  slicing_specs {}  # overall slice
  slicing_specs {
    feature_keys: ["race"]
  }
  slicing_specs {
    feature_keys: ["sex"]
  }
""", tfma.EvalConfig())

In [39]:
eval_config_2 = tfma.EvalConfig(
        model_specs=[tfma.ModelSpec(label_key="label")],
        slicing_specs=[
            tfma.SlicingSpec(),
            tfma.SlicingSpec(feature_keys=["race"]),
            tfma.SlicingSpec(feature_keys=["sex"]),
        ],
        metrics_specs=[
            tfma.MetricsSpec(
                metrics=[
                    tfma.MetricConfig(class_name="ExampleCount"),
                    tfma.MetricConfig(class_name="BinaryAccuracy"),
                    tfma.MetricConfig(class_name="BinaryCrossentropy"),
                    tfma.MetricConfig(class_name="AUC"),
                    tfma.MetricConfig(class_name="AUCPrecisionRecall"),
                    tfma.MetricConfig(class_name="Precision"),
                    tfma.MetricConfig(class_name="Recall"),
                    tfma.MetricConfig(class_name="MeanLabel"),
                    tfma.MetricConfig(class_name="MeanPrediction"),
                    tfma.MetricConfig(class_name="Calibration"),
                    tfma.MetricConfig(class_name="CalibrationPlot"),
                    tfma.MetricConfig(class_name="ConfusionMatrixPlot"),
                ],
                thresholds={
                    "BinaryAccuracy": tfma.config.MetricThreshold(
                        value_threshold=tfma.GenericValueThreshold(
                            lower_bound={"value": 0.5}
                        ),
                        change_threshold=tfma.GenericChangeThreshold(
                            direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                            absolute={"value": 0.01},
                        ),
                    )
                },
            )
        ],
    )

## Resolve latest blessed model

In [40]:
from tfx.dsl.components.common.resolver import Resolver
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import LatestBlessedModelStrategy
from tfx.types import Channel
from tfx.types.standard_artifacts import Model, ModelBlessing

# Setup the Resolver node to find the latest blessed model
model_resolver = Resolver(
      strategy_class=LatestBlessedModelStrategy,
      model=Channel(type=Model),
      model_blessing=Channel(
          type=ModelBlessing)).with_id(
              'latest_blessed_model_resolver')

In [41]:
# Run the Resolver node
context.run(model_resolver)

ExecutionResult(
    component_id: latest_blessed_model_resolver
    execution_id: 8
    outputs:
        model: Channel(
            type_name: Model
            artifacts: []
            additional_properties: {}
            additional_custom_properties: {}
        )
        model_blessing: Channel(
            type_name: ModelBlessing
            artifacts: []
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [42]:
model_resolver.outputs['model']

.type_name,Model
._artifacts,[]


## Run the Evaluator component

In [43]:
from tfx.components import Evaluator
evaluator = Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    baseline_model=model_resolver.outputs['model'],
    eval_config=eval_config_1)

In [44]:
context.run(evaluator, enable_cache=False)

ERROR:absl:There are change thresholds, but the baseline is missing. This is allowed only when rubber stamping (first run).
Exception ignored in: <function CapturableResource.__del__ at 0x000001F590D5E950>
Traceback (most recent call last):
  File "Z:\ProgramData\Anaconda3\envs\tfx_env\lib\site-packages\tensorflow\python\training\tracking\tracking.py", line 277, in __del__
    self._destroy_resource()
  File "Z:\ProgramData\Anaconda3\envs\tfx_env\lib\site-packages\tensorflow\python\eager\def_function.py", line 889, in __call__
    result = self._call(*args, **kwds)
  File "Z:\ProgramData\Anaconda3\envs\tfx_env\lib\site-packages\tensorflow\python\eager\def_function.py", line 924, in _call
    results = self._stateful_fn(*args, **kwds)
  File "Z:\ProgramData\Anaconda3\envs\tfx_env\lib\site-packages\tensorflow\python\eager\function.py", line 3022, in __call__
    filtered_flat_args) = self._maybe_define_function(args, kwargs)
  File "Z:\ProgramData\Anaconda3\envs\tfx_env\lib\site-packages

    out = weak_wrapped_fn().__wrapped__(*args, **kwds)
AttributeError: 'NoneType' object has no attribute '__wrapped__'
Exception ignored in: <function CapturableResource.__del__ at 0x000001F590D5E950>
Traceback (most recent call last):
  File "Z:\ProgramData\Anaconda3\envs\tfx_env\lib\site-packages\tensorflow\python\training\tracking\tracking.py", line 277, in __del__
    self._destroy_resource()
  File "Z:\ProgramData\Anaconda3\envs\tfx_env\lib\site-packages\tensorflow\python\eager\def_function.py", line 889, in __call__
    result = self._call(*args, **kwds)
  File "Z:\ProgramData\Anaconda3\envs\tfx_env\lib\site-packages\tensorflow\python\eager\def_function.py", line 924, in _call
    results = self._stateful_fn(*args, **kwds)
  File "Z:\ProgramData\Anaconda3\envs\tfx_env\lib\site-packages\tensorflow\python\eager\function.py", line 3022, in __call__
    filtered_flat_args) = self._maybe_define_function(args, kwargs)
  File "Z:\ProgramData\Anaconda3\envs\tfx_env\lib\site-packages\ten

ExecutionResult(
    component_id: Evaluator
    execution_id: 9
    outputs:
        evaluation: Channel(
            type_name: ModelEvaluation
            artifacts: [Artifact(artifact: id: 16
        type_id: 22
        uri: "tfx\\Evaluator\\evaluation\\9"
        custom_properties {
          key: "name"
          value {
            string_value: "evaluation"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Evaluator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.0.0"
          }
        }
        state: LIVE
        , artifact_type: id: 22
        name: "ModelEvaluation"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        blessing: Channel(
            type_name: ModelBlessing
            artifacts: [Artifact(artifact: id: 17
        type_id: 23
        uri: "tfx\\Evaluator\\blessing\\9"
        custom_properties {
          key: "blessed"
          value {
            int_value: 1
          }
        }
        custom_properties {
          key: "current_model"
          value {
            string_value: "tfx\\Trainer\\model\\7"
          }
        }
        custom_properties {
          key: "current_model_id"
          value {
            int_value: 14
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "blessing"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Evaluator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.0.0"
          }
        }
        state: LIVE
        , artifact_type: id: 23
        name: "ModelBlessing"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [45]:
# Print component output keys
evaluator.outputs.keys()

dict_keys(['evaluation', 'blessing'])

In [46]:
# Get `Evaluator` blessing output URI
blessing_uri = evaluator.outputs['blessing'].get()[0].uri

# List files under URI
os.listdir(blessing_uri)

['BLESSED']

In [47]:
# Visualize the evaluation results
context.show(evaluator.outputs['evaluation'])

## Model Anlysis

In [48]:
import tensorflow_model_analysis as tfma

# Get the TFMA output result path and load the result.
PATH_TO_RESULT = evaluator.outputs['evaluation'].get()[0].uri
tfma_result = tfma.load_eval_result(PATH_TO_RESULT)

# Show data sliced along feature column trip_start_hour.
tfma.view.render_slicing_metrics(
    tfma_result, slicing_column='sex')

SlicingMetricsViewer(config={'weightedExamplesColumn': 'example_count'}, data=[{'slice': 'sex:Male', 'metrics'…

In [49]:
# Get `evaluation` output URI
PATH_TO_RESULT = evaluator.outputs['evaluation'].get()[0].uri

# Print validation result
print(tfma.load_validation_result(PATH_TO_RESULT))

validation_ok: true
validation_details {
  slicing_details {
    slicing_spec {
    }
    num_matching_slices: 8
  }
}



In [50]:
# Re-run the Resolver component
context.run(model_resolver)

ExecutionResult(
    component_id: latest_blessed_model_resolver
    execution_id: 10
    outputs:
        model: Channel(
            type_name: Model
            artifacts: [Artifact(artifact: id: 14
        type_id: 18
        uri: "tfx\\Trainer\\model\\7"
        custom_properties {
          key: "name"
          value {
            string_value: "model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.0.0"
          }
        }
        state: LIVE
        create_time_since_epoch: 1628071120220
        last_update_time_since_epoch: 1628071140590
        , artifact_type: id: 18
        name: "Model"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        model_blessing: Channel(
            type_name: ModelBlessing
            artifacts: [Artifact(artifact: id: 17
        type_id: 23
        uri: "tfx\\Evaluator\\blessing\\9"
        custom_properties {
          key: "blessed"
          value {
            int_value: 1
          }
        }
        custom_properties {
          key: "current_model"
          value {
            string_value: "tfx\\Trainer\\model\\7"
          }
        }
        custom_properties {
          key: "current_model_id"
          value {
            int_value: 14
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "blessing"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Evaluator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.0.0"
          }
        }
        state: LIVE
        create_time_since_epoch: 1628071141088
        last_update_time_since_epoch: 1628071174220
        , artifact_type: id: 23
        name: "ModelBlessing"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [51]:
# Get path to latest blessed model
model_resolver.outputs['model'].get()[0].uri

'tfx\\Trainer\\model\\7'

## Comparing two models

Now let's see how `Evaluator` compares two models. You will train the same model with more epochs and this should hopefully result in higher accuracy and overall metrics.

In [52]:
# from tfx.proto import trainer_pb2

# # Setup trainer to train with more epochs
# trainer = Trainer(
#     module_file=os.path.abspath(_trainer_module_file),
#     examples=transform.outputs['transformed_examples'],
#     transform_graph=transform.outputs['transform_graph'],
#     hyperparameters=tuner.outputs['best_hyperparameters'],
#     schema=schema_gen.outputs['schema'],
#     train_args=trainer_pb2.TrainArgs(num_steps=500),
#     eval_args=trainer_pb2.EvalArgs(num_steps=200))

# # Run trainer
# context.run(trainer, enable_cache=False)

In [53]:
# # Re-run the evaluator
# evaluator = Evaluator(
#     examples=example_gen.outputs['examples'],
#     model=trainer.outputs['model'],
#     baseline_model=model_resolver.outputs['model'],
#     eval_config=eval_config_1)

# context.run(evaluator, enable_cache=False)

In [54]:
# Re-run the resolver
#context.run(model_resolver, enable_cache=False)

In [55]:
# Get path to latest blessed model
#model_resolver.outputs['model'].get()[0].uri

In [56]:
#context.show(evaluator.outputs['evaluation'])

# Pusher

In [57]:
from tfx.components import Pusher
from tfx.proto import pusher_pb2

In [58]:
pusher_kwargs = {
    "model": trainer.outputs["model"],
    "model_blessing": evaluator.outputs["blessing"],
    "push_destination": pusher_pb2.PushDestination(
        filesystem=pusher_pb2.PushDestination.Filesystem(
            base_directory=serving_model_dir
        )
    )
}

In [59]:
pusher = Pusher(**pusher_kwargs)

In [60]:
context.run(pusher)

ExecutionResult(
    component_id: Pusher
    execution_id: 11
    outputs:
        pushed_model: Channel(
            type_name: PushedModel
            artifacts: [Artifact(artifact: id: 18
        type_id: 25
        uri: "tfx\\Pusher\\pushed_model\\11"
        custom_properties {
          key: "name"
          value {
            string_value: "pushed_model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Pusher"
          }
        }
        custom_properties {
          key: "pushed"
          value {
            int_value: 1
          }
        }
        custom_properties {
          key: "pushed_destination"
          value {
            string_value: "Z:\\tfx\\tfx\\serving_model\\1628071176"
          }
        }
        custom_properties {
          key: "pushed_version"
          value {
            string_value: "1628071176"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.0.0"
          }
        }
        state: LIVE
        , artifact_type: id: 25
        name: "PushedModel"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))